```
havoc x, y;
if (x > 0 and y < 0)
  x = x + 1
else
  y = y - 1
assume y > x ; 
assert y > x + 100 ;
while y > x do
   havoc x ;
   y = y + x
```

In [16]:
import z3

In [22]:
# execute: havoc x, y
env = dict()
pc = list()
(env, pc)

({}, [])

In [25]:
# symex: havoc x, y on state (env, pc)
env['x'] = z3.FreshInt('R')
env['y'] = z3.FreshInt('Y')
(env, pc)

({'x': R!5, 'y': Y!6}, [])

In [26]:
r = z3.And(env['x'] > 0 , env['y'] < 0)
r

And(R!5 > 0, Y!6 < 0)

In [27]:
# check that can execute then-branch of if statement
s = z3.Solver()
s.add(pc)
s.add(r)
s.check()
# expect: yes

sat

In [29]:
# check that can execute else-branch of if statement
s = z3.Solver()
s.add(pc)
s.add(z3.Not(r))
s.check()
# expect: yes

sat

In [30]:
env1 = env.copy()
pc1 = pc.copy()
env2 = env.copy()
pc2 = pc.copy()

In [31]:
pc1.append(r)
pc2.append(z3.Not(r))

In [32]:
env1['x'] = env1['x'] + z3.IntVal(1)
env2['y'] = env2['y'] - z3.IntVal(1)

In [33]:
# two new symbolic states to execute after the if statement
print((env1, pc1))
print((env2, pc2))

({'x': R!5 + 1, 'y': Y!6}, [And(R!5 > 0, Y!6 < 0)])
({'x': R!5, 'y': Y!6 - 1}, [Not(And(R!5 > 0, Y!6 < 0))])


In [34]:
pc1.append(env1['y'] > env1['x'])
pc2.append(env2['y'] > env2['x'])
(pc1, pc2)

([And(R!5 > 0, Y!6 < 0), Y!6 > R!5 + 1],
 [Not(And(R!5 > 0, Y!6 < 0)), Y!6 - 1 > R!5])

In [35]:
s = z3.Solver()
s.add(pc1)
s.check()

unsat

In [36]:
s.reset()
s.add(pc2)
s.check()

sat

In [37]:
r = env2['y'] > env2['x'] + z3.IntVal(100)

In [38]:
r

Y!6 - 1 > R!5 + 100

In [39]:
s.reset()
s.add(pc2)
s.add(z3.Not(r))
s.check()

sat

In [40]:
s.model()

[R!5 = -3, Y!6 = -1]

In [41]:
print("error: assert can fail")

error: assert can fail


In [42]:
s.reset()
s.add(pc2)
s.add(r)
s.check()

sat

In [43]:
s.model()

[R!5 = -103, Y!6 = -1]

In [45]:
pc2.append(r)
(env2, pc2)

({'x': R!5, 'y': Y!6 - 1},
 [Not(And(R!5 > 0, Y!6 < 0)), Y!6 - 1 > R!5, Y!6 - 1 > R!5 + 100])

In [46]:
r = env2['y'] > env2['x']
s.reset()
s.add(pc2)
s.add(z3.Not(r))
s.check()

unsat

In [47]:
s.reset()
s.add(pc2)
s.add(r)
s.check()


sat

In [49]:
pc2.append(r)

In [50]:
env2['x'] = z3.FreshInt('X')
env2['y'] = env2['y'] + env2['x']
(env2, pc2)

({'x': X!14, 'y': Y!6 - 1 + X!14},
 [Not(And(R!5 > 0, Y!6 < 0)),
  Y!6 - 1 > R!5,
  Y!6 - 1 > R!5 + 100,
  Y!6 - 1 > R!5])

In [51]:
r = env2['y'] > env2['x']
s.reset()
s.add(pc2)
s.add(z3.Not(r))
s.check()

sat

In [52]:
r = env2['y'] > env2['x']
s.reset()
s.add(pc2)
s.add(r)
s.check()

sat

In [54]:
env3 = env2.copy()
pc3 = pc2.copy()
pc3.append(z3.Not(r))
(env3, pc3)

({'x': X!14, 'y': Y!6 - 1 + X!14},
 [Not(And(R!5 > 0, Y!6 < 0)),
  Y!6 - 1 > R!5,
  Y!6 - 1 > R!5 + 100,
  Y!6 - 1 > R!5,
  Not(Y!6 - 1 + X!14 > X!14)])

In [55]:
pc2.append(r)
env2['x'] = z3.FreshInt('X')
env2['y'] = env2['y'] + env2['x']
(env2, pc2)

({'x': X!15, 'y': Y!6 - 1 + X!14 + X!15},
 [Not(And(R!5 > 0, Y!6 < 0)),
  Y!6 - 1 > R!5,
  Y!6 - 1 > R!5 + 100,
  Y!6 - 1 > R!5,
  Y!6 - 1 + X!14 > X!14])

In [56]:
r = env2['y'] > env2['x']
s.reset()
s.add(pc2)
s.add(z3.Not(r))
s.check()

sat

In [57]:
env4 = env2.copy()
pc4 = pc2.copy()
pc4.append(z3.Not(r))
(env4, pc4)

({'x': X!15, 'y': Y!6 - 1 + X!14 + X!15},
 [Not(And(R!5 > 0, Y!6 < 0)),
  Y!6 - 1 > R!5,
  Y!6 - 1 > R!5 + 100,
  Y!6 - 1 > R!5,
  Y!6 - 1 + X!14 > X!14,
  Not(Y!6 - 1 + X!14 + X!15 > X!15)])

In [58]:
(env1, pc1), (env2, pc2), (env3, pc3), (env4, pc4)

(({'x': R!5 + 1, 'y': Y!6}, [And(R!5 > 0, Y!6 < 0), Y!6 > R!5 + 1]),
 ({'x': X!15, 'y': Y!6 - 1 + X!14 + X!15},
  [Not(And(R!5 > 0, Y!6 < 0)),
   Y!6 - 1 > R!5,
   Y!6 - 1 > R!5 + 100,
   Y!6 - 1 > R!5,
   Y!6 - 1 + X!14 > X!14]),
 ({'x': X!14, 'y': Y!6 - 1 + X!14},
  [Not(And(R!5 > 0, Y!6 < 0)),
   Y!6 - 1 > R!5,
   Y!6 - 1 > R!5 + 100,
   Y!6 - 1 > R!5,
   Not(Y!6 - 1 + X!14 > X!14)]),
 ({'x': X!15, 'y': Y!6 - 1 + X!14 + X!15},
  [Not(And(R!5 > 0, Y!6 < 0)),
   Y!6 - 1 > R!5,
   Y!6 - 1 > R!5 + 100,
   Y!6 - 1 > R!5,
   Y!6 - 1 + X!14 > X!14,
   Not(Y!6 - 1 + X!14 + X!15 > X!15)]))

In [60]:
print(pc2)
s.reset()
s.add(pc2)
s.check()
s.model()

[Not(And(R!5 > 0, Y!6 < 0)), Y!6 - 1 > R!5, Y!6 - 1 > R!5 + 100, Y!6 - 1 > R!5, Y!6 - 1 + X!14 > X!14]


[R!5 = 1, Y!6 = 103]